# Predictive Models

## "Housekeeping"

In [1]:
import json

import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.metrics import (
    make_scorer,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.svm import SVR
from sklearn.utils import shuffle
from tqdm import tqdm as progress_bar

from utils import (
    CONTINUOUS_VARIABLES,
    DISCRETE_VARIABLES,
    NOMINAL_VARIABLES,
    ORDINAL_VARIABLES,
    TARGET_VARIABLES,
    bias_score,
    encode_ordinals,
    load_clean_data,
    max_deviation,
)

In [2]:
random_state = np.random.RandomState(42)

In [3]:
pd.set_option("display.max_columns", 250)

## Load the Data

### 1. Original Data

The DataFrame `df1` holds the cleaned data from notebook 1 with the all the nominal and ordinal features automatically translated to factor variables and ordered integer values.

In [4]:
df1 = load_clean_data("data/data_clean.csv")

This cell basically **replaces** all the manual work that went into generating new and identifying "interesting" features in notebooks 2 and 3.

In [5]:
df1 = pd.concat([
    df1[CONTINUOUS_VARIABLES + DISCRETE_VARIABLES + ORDINAL_VARIABLES + TARGET_VARIABLES],
    pd.get_dummies(df1[NOMINAL_VARIABLES], dtype=int),
], axis=1)
# Re-order the columns for convenience.
df1 = df1[sorted(set(df1.columns) - set(TARGET_VARIABLES)) + TARGET_VARIABLES]

In [6]:
df1 = encode_ordinals(df1)

In [7]:
df1 = shuffle(df1, random_state=random_state)

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2898 entries, (144, 535153070) to (867, 907253130)
Columns: 248 entries, 1st Flr SF to SalePrice
dtypes: float64(19), int64(229)
memory usage: 5.5 MB


In [9]:
df1.head()

,,1st Flr SF,2nd Flr SF,3Ssn Porch,Alley_Grvl,Alley_NA,Alley_Pave,Bedroom AbvGr,Bldg Type_1Fam,Bldg Type_2FmCon,Bldg Type_Duplx,Bldg Type_TwnhsE,Bldg Type_TwnhsI,Bsmt Cond,Bsmt Exposure,Bsmt Full Bath,Bsmt Half Bath,Bsmt Qual,Bsmt Unf SF,BsmtFin SF 1,BsmtFin SF 2,BsmtFin Type 1,BsmtFin Type 2,Central Air_N,Central Air_Y,Condition 1_Artery,Condition 1_Feedr,Condition 1_Norm,Condition 1_PosA,Condition 1_PosN,Condition 1_RRAe,Condition 1_RRAn,Condition 1_RRNe,Condition 1_RRNn,Condition 2_Artery,Condition 2_Feedr,Condition 2_Norm,Condition 2_PosA,Condition 2_PosN,Condition 2_RRAe,Condition 2_RRAn,Condition 2_RRNe,Condition 2_RRNn,Electrical,Enclosed Porch,Exter Cond,Exter Qual,Exterior 1st_AsbShng,Exterior 1st_AsphShn,Exterior 1st_BrkComm,Exterior 1st_BrkFace,Exterior 1st_CBlock,Exterior 1st_CemntBd,Exterior 1st_HdBoard,Exterior 1st_ImStucc,Exterior 1st_MetalSd,Exterior 1st_Other,Exterior 1st_Plywood,Exterior 1st_PreCast,Exterior 1st_Stone,Exterior 1st_Stucco,Exterior 1st_VinylSd,Exterior 1st_Wd Sdng,Exterior 1st_WdShing,Exterior 2nd_AsbShng,Exterior 2nd_AsphShn,Exterior 2nd_BrkComm,Exterior 2nd_BrkFace,Exterior 2nd_CBlock,Exterior 2nd_CemntBd,Exterior 2nd_HdBoard,Exterior 2nd_ImStucc,Exterior 2nd_MetalSd,Exterior 2nd_Other,Exterior 2nd_Plywood,Exterior 2nd_PreCast,Exterior 2nd_Stone,Exterior 2nd_Stucco,Exterior 2nd_VinylSd,Exterior 2nd_Wd Sdng,Exterior 2nd_WdShing,Fence,Fireplace Qu,Fireplaces,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Full Bath,Functional,Garage Area,Garage Cars,Garage Cond,Garage Finish,Garage Qual,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Type_NA,Gr Liv Area,Half Bath,Heating QC,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,House Style_1.5Fin,House Style_1.5Unf,House Style_1Story,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Kitchen AbvGr,Kitchen Qual,Land Contour_Bnk,Land Contour_HLS,Land Contour_Low,Land Contour_Lvl,Land Slope,Lot Area,Lot Config_Corner,Lot Config_CulDSac,Lot Config_FR2,Lot Config_FR3,Lot Config_Inside,Lot Shape,Low Qual Fin SF,MS SubClass_020,MS SubClass_030,MS SubClass_040,MS SubClass_045,MS SubClass_050,MS SubClass_060,MS SubClass_070,MS SubClass_075,MS SubClass_080,MS SubClass_085,MS SubClass_090,MS SubClass_120,MS SubClass_150,MS SubClass_160,MS SubClass_180,MS SubClass_190,MS Zoning_A,MS Zoning_C,MS Zoning_FV,MS Zoning_I,MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,MS Zoning_RP,Mas Vnr Area,Mas Vnr Type_BrkCmn,Mas Vnr Type_BrkFace,Mas Vnr Type_CBlock,Mas Vnr Type_None,Mas Vnr Type_Stone,Misc Feature_Elev,Misc Feature_Gar2,Misc Feature_NA,Misc Feature_Othr,Misc Feature_Shed,Misc Feature_TenC,Misc Val,Mo Sold,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_Names,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Open Porch SF,Overall Cond,Overall Qual,Paved Drive,Pool Area,Pool QC,Roof Matl_ClyTile,Roof Matl_CompShg,Roof Matl_Membran,Roof Matl_Metal,Roof Matl_Roll,Roof Matl_Tar&Grv,Roof Matl_WdShake,Roof Matl_WdShngl,Roof Style_Flat,Roof Style_Gable,Roof Style_Gambrel,Roof Style_Hip,Roof Style_Mansard,Roof Style_Shed,Sale Condition_Abnorml,Sale Condition_AdjLand,Sale Condition_Alloca,Sale Condition_Family,Sale Condition_Normal,Sale Condition_Partial,Sale Type_COD,Sale Type_CWD,Sale Type_Con,Sale Type_ConLD,Sale Type_ConLI,Sale Type_ConLw,Sale Type_New,Sale Type_Oth,Sale Type_VWD,Sale Type_WD,Screen Porch,Street_G

Obtain the raw numpy arrays:

In [10]:
X1 = df1.drop(columns="SalePrice").values
y1 = df1["SalePrice"].values

### 2. Improved Data

The DataFrame `df2` holds the data as manually processed in notebooks 2 and 3.

New features like the *years_since_\** columns were generated or derived from other variables like *has 2nd Flr* (from the continuous *2nd Flr SF*). Further, factor variables were created taking into account patterns in the visualizations. For example, *Bldg Type*'s (from `df1`) five categories were condensed into just three. In summary, `df2` has less than half as many dimensions as `df1` to accomodate for a potential curse of dimensionality.

In [11]:
df2 = load_clean_data("data/data_clean_with_transformations_and_factors.csv")

In [12]:
df2 = encode_ordinals(df2)

In [13]:
df2 = shuffle(df2, random_state=random_state)

In [14]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2883 entries, (2775, 907175060) to (2659, 902305090)
Columns: 109 entries, 1st Flr SF to SalePrice (box-cox-0)
dtypes: float64(27), int64(82)
memory usage: 2.5 MB


In [15]:
df2.head()

,,1st Flr SF,1st Flr SF (box-cox-0),2nd Flr SF,3Ssn Porch,Bedroom AbvGr,Bsmt Cond,Bsmt Exposure,Bsmt Full Bath,Bsmt Half Bath,Bsmt Qual,Bsmt Unf SF,BsmtFin SF 1,BsmtFin SF 2,BsmtFin Type 1,BsmtFin Type 2,Electrical,Enclosed Porch,Fence,Fireplace Qu,Fireplaces,Full Bath,Functional,Garage Area,Garage Cars,Garage Cond,Garage Finish,Garage Qual,Gr Liv Area,Gr Liv Area (box-cox-0),Half Bath,Kitchen AbvGr,Kitchen Qual,Land Slope,Lot Area,Lot Area (box-cox-0.1),Lot Shape,Low Qual Fin SF,Mas Vnr Area,Misc Val,Mo Sold,Open Porch SF,Overall Cond,Overall Qual,Paved Drive,Pool Area,Pool QC,Screen Porch,TotRms AbvGrd,Total Bath,Total Bsmt SF,Total Porch SF,Total SF,Total SF (box-cox-0.2),Utilities,Wood Deck SF,abnormal_sale,air_cond,build_type_1Fam,build_type_2Fam,build_type_Twnhs,found_BrkTil,found_CBlock,found_PConc,has 2nd Flr,has Bsmt,has Fireplace,has Garage,has Pool,has Porch,major_street,new_home,nhood_Blmngtn,nhood_Blueste,nhood_BrDale,nhood_BrkSide,nhood_ClearCr,nhood_CollgCr,nhood_Crawfor,nhood_Edwards,nhood_Gilbert,nhood_Greens,nhood_GrnHill,nhood_IDOTRR,nhood_Landmrk,nhood_MeadowV,nhood_Mitchel,nhood_NPkVill,nhood_NWAmes,nhood_Names,nhood_NoRidge,nhood_NridgHt,nhood_OldTown,nhood_SWISU,nhood_Sawyer,nhood_SawyerW,nhood_Somerst,nhood_StoneBr,nhood_Timber,nhood_Veenker,park,partial_sale,railway,recently_built,recently_remodeled,remodeled,years_since_built,years_since_remodeled,SalePrice,SalePrice (box-cox-0)
Order,PID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2775,907175060,1494.0,7.309212,0.0,0.0,3,3,4,1,0,4,279.0,1200.0,0.0,6,1,4,0.0,0,0,0,2,7,576.0,2,3,2,3,1494.0,7.309212,0,1,3,2,9236.0,14.920020,2,0.0,0.0,0.0,7,27.0,4,5,2,0.0,0,0.0,6,3.0,1479.0,195.0,2973.0,19.751925,3,168.0,0,1,1,0,0,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,9,9,217000.0,12.287653
2139,907202100,907.0,6.810142,0.0,0.0,3,3,3,1,0,4,399.0,60.0,417.0,3,4,4,0.0,3,0,0,1,7,308.0,1,3,2,3,907.0,6.810142,0,1,2,2,16300.0,16.376597,2,0.0,0.0,0.0,1,0.0,3,4,2,0.0,0,0.0,5,2.0,876.0,0.0,1783.0,17.346025,3,0.0,0,1,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,30,130000.0,11.775290
1044,527451290,483.0,6.180017,504.0,0.0,2,3,1,0,0,3,96.0,387.0,0.0,5,1,4,0.0,0,0,0,1,7,264.0,1,3,1,3,987.0,6.894670,1,1,2,2,1680.0,11.015074,3,0.0,232.0,0.0,7,0.0,4,5,2,0.0,0,0.0,4,1.5,483.0,0.0,1470.0,16.499753,3,0.0,1,1,0,0,1,0,1,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37,37,85400.0,11.355101
643,535303110,1062.0,6.967909,0.0,0.0,3,3,1,1,0,3,572.0,490.0,0.0,5,1,4,0.0,0,0,0,1,7,297.0,1,3,1,3,1062.0,6.967909,0,1,2,2,8128.0,14.603584,2,0.0,80.0,0.0,2,0.0,6,5,2,0.0,0,0.0,6,2.0,1062.0,0.0,2124.0,18.142001,3,0.0,0,1,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,55,55,131000.0,11.782953
160,535401080,1287.0,7.160069,0.0,0.0,3,3,1,1,0,3,733.0,72.0,258.0,5,3,4,0.0,0,4,1,1,7,576.0,2,3,3,3,1287.0,7.160069,0,1,3,2,9830.0,15.075832,2,0.0,0.0,0.0,3,17.0,6,4,2,0.0,0,182.0,7,2.0,1063.0,563.0,2350.0,18.614762,3,364.0,0,1,1,0,0,0,1,0,0,1,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,51,4,162000.0,11.995352


Obtain the raw numpy arrays:

In [16]:
X2 = df2.drop(columns=["SalePrice", "SalePrice (box-cox-0)"]).values
y2 = df2["SalePrice"].values
y2l = df2["SalePrice (box-cox-0)"].values

### 3. Manual Feature Pre-Selection

Also, notebook 2 collects variables that correlate either weakly ($0.33 < \vert\rho\vert < 0.66$) or strongly ($\vert\rho\vert > 0.66$) with the *SalePrice* (or the logarithm thereof). These variables serve as a "naive" feature pre-selection.

In [17]:
with open("data/correlated_variables.json", "r") as file:
    _ = json.loads(file.read())
    weakly_correlated = _["weakly_correlated"]
    strongly_correlated = _["strongly_correlated"]

In [18]:
pre_selection = sorted(set(weakly_correlated + strongly_correlated) & set(df2.columns))

The `df3` DataFrame is just a subset of `df2` (71 columns).

In [19]:
df3 = df2[pre_selection + TARGET_VARIABLES]

In [20]:
df3.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2883 entries, (2775, 907175060) to (2659, 902305090)
Columns: 32 entries, 1st Flr SF to SalePrice (box-cox-0)
dtypes: float64(16), int64(16)
memory usage: 777.2 KB


In [21]:
df3.head()

,,1st Flr SF,1st Flr SF (box-cox-0),Bsmt Exposure,Bsmt Qual,BsmtFin SF 1,BsmtFin Type 1,Fireplace Qu,Fireplaces,Full Bath,Garage Area,Garage Cars,Garage Cond,Garage Finish,Garage Qual,Gr Liv Area,Gr Liv Area (box-cox-0),Half Bath,Kitchen Qual,Lot Area (box-cox-0.1),Lot Shape,Mas Vnr Area,Overall Qual,Paved Drive,TotRms AbvGrd,Total Bath,Total Bsmt SF,Total Porch SF,Total SF,Total SF (box-cox-0.2),Wood Deck SF,SalePrice,SalePrice (box-cox-0)
Order,PID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2775,907175060,1494.0,7.309212,4,4,1200.0,6,0,0,2,576.0,2,3,2,3,1494.0,7.309212,0,3,14.920020,2,0.0,5,2,6,3.0,1479.0,195.0,2973.0,19.751925,168.0,217000.0,12.287653
2139,907202100,907.0,6.810142,3,4,60.0,3,0,0,1,308.0,1,3,2,3,907.0,6.810142,0,2,16.376597,2,0.0,4,2,5,2.0,876.0,0.0,1783.0,17.346025,0.0,130000.0,11.775290
1044,527451290,483.0,6.180017,1,3,387.0,5,0,0,1,264.0,1,3,1,3,987.0,6.894670,1,2,11.015074,3,232.0,5,2,4,1.5,483.0,0.0,1470.0,16.499753,0.0,85400.0,11.355101
643,535303110,1062.0,6.967909,1,3,490.0,5,0,0,1,297.0,1,3,1,3,1062.0,6.967909,0,2,14.603584,2,80.0,5,2,6,2.0,1062.0,0.0,2124.0,18.142001,0.0,131000.0,11.782953
160,535401080,1287.0,7.160069,1,3,72.0,5,4,1,1,576.0,2,3,3,3,1287.0,7.160069,0,3,15.075832,2,0.0,4,2,7,2.0,1063.0,563.0,2350.0,18.614762,364.0,162000.0,11.995352


Obtain the raw numpy arrays:

In [22]:
X3 = df3.drop(columns=["SalePrice", "SalePrice (box-cox-0)"]).values
y3 = df3["SalePrice"].values
y3l = df3["SalePrice (box-cox-0)"].values

## Re-usable Components

Define a function to run k-fold cross validation to obtain unbiased estimators for the following scores / errors:
- Bias
- Mean Absolute Error
- Maximum Deviation (just to see the worst case prediction of a model)
- R2 (coefficient of determination)
- Root Mean Squared Error (default for comparison)

In [23]:
def cross_validation(X, y, *, model, k=10, log=False, desc=None):
    """Perform a k-fold cross validation."""
    bias, mae, max_dev, r2, rmse = [], [], [], [], []
    # Iterate over the k folds.
    for train, test in progress_bar(KFold(n_splits=k).split(X), desc=desc, total=k):
        model.fit(X[train], y[train])
        y_pred = model.predict(X[test])
        # If the sales price is provided on a log scale,
        # take the exponent first so that scores and
        # errors are comparable to the non-logged counterparts.
        if log:
            y_true, y_pred = np.exp(y[test]), np.exp(y_pred) 
        else:
            y_true, y_pred = y[test], y_pred
        # Collect the scores/errors for each fold.
        bias.append(bias_score(y_true, y_pred))
        mae.append(mean_absolute_error(y_true, y_pred))
        max_dev.append(max_deviation(y_true, y_pred))
        r2.append(r2_score(y_true, y_pred))
        rmse.append(mean_squared_error(y_true, y_pred))
    # Round for convenience.
    return {
        "bias": np.round(np.mean(bias)),
        "mae": np.round(np.mean(mae)),
        "max_dev": np.round(np.mean(max_dev)),
        "r2": np.round(np.mean(r2), 3),
        "rmse": np.round(np.sqrt(np.mean(rmse))),
    }

Use one dictionary to store all the results in a systematic way.

In [24]:
results = {}

## Linear Regression

A plain OLS regression model serves as the base case for benchmarking.

In [25]:
lm = LinearRegression()

### 1. Original Data

Given the unprocessed data, the linear model is not able to make a good fit at all.

In [26]:
results[('lm','o')] = cross_validation(X1, y1, model=lm)
results[('lm','o')]

100%|██████████| 10/10 [00:00<00:00, 12.35it/s]


{'bias': 39452975.0,
 'mae': 44610752.0,
 'max_dev': 11521546650.0,
 'r2': -568953281.52,
 'rmse': 1902925384.0}

### 2. Improved Data

#### a) Normal Scale

In [27]:
results[('lm','i')] = cross_validation(X2, y2, model=lm)
results[('lm','i')]

100%|██████████| 10/10 [00:00<00:00, 23.02it/s]


{'bias': -41.0,
 'mae': 15219.0,
 'max_dev': 130832.0,
 'r2': 0.917,
 'rmse': 21864.0}

#### b) Log Scale

In [28]:
results[('lm','il')] = cross_validation(X2, y2l, model=lm, log=True)
results[('lm','il')]

100%|██████████| 10/10 [00:00<00:00, 35.00it/s]


{'bias': -859.0,
 'mae': 12789.0,
 'max_dev': 119951.0,
 'r2': 0.936,
 'rmse': 19223.0}

### 3. Improved Data with pre-selected Features

#### a) Normal Scale

In [29]:
results[('lm','p')] = cross_validation(X3, y3, model=lm)
results[('lm','p')]

100%|██████████| 10/10 [00:00<00:00, 189.50it/s]


{'bias': -17.0,
 'mae': 18418.0,
 'max_dev': 151335.0,
 'r2': 0.887,
 'rmse': 25704.0}

#### b) Log Scale

In [30]:
results[('lm','pl')] = cross_validation(X3, y3l, model=lm, log=True)
results[('lm','pl')]

100%|██████████| 10/10 [00:00<00:00, 337.94it/s]


{'bias': -1356.0,
 'mae': 16148.0,
 'max_dev': 144356.0,
 'r2': 0.907,
 'rmse': 23341.0}

## LASSO

In [31]:
tol = 0.1
grid_search = GridSearchCV(
    estimator=Lasso(tol=tol, selection="random", random_state=random_state),
    param_grid={"alpha": [2 ** x for x in range(-8, 4)] + list(range(12, 65, 4))},
    cv=KFold(n_splits=4),
    n_jobs=-1,
)

### 1. Original Data

In [32]:
grid_search.fit(X1, y1)
alpha = grid_search.best_params_["alpha"]
alpha

20

In [33]:
results[("lasso", "o")] = cross_validation(X1, y1, model=Lasso(alpha=alpha, tol=tol))
results[("lasso", "o")]

100%|██████████| 10/10 [00:00<00:00, 26.11it/s]


{'bias': 278.0,
 'mae': 20892.0,
 'max_dev': 269314.0,
 'r2': 0.817,
 'rmse': 33496.0}

### 2. Improved Data

#### a) Normal Scale

In [34]:
grid_search.fit(X2, y2)
alpha = grid_search.best_params_["alpha"]
alpha

64

In [35]:
results[("lasso", "i")] = cross_validation(X2, y2, model=Lasso(alpha=alpha, tol=tol))
results[("lasso", "i")]

100%|██████████| 10/10 [00:00<00:00, 60.29it/s]


{'bias': -28.0,
 'mae': 17483.0,
 'max_dev': 142797.0,
 'r2': 0.896,
 'rmse': 24443.0}

#### b) Log Scale

In [36]:
grid_search.fit(X2, y2l)
alpha = grid_search.best_params_["alpha"]
alpha

0.00390625

In [37]:
results[("lasso", "il")] = cross_validation(X2, y2l, model=Lasso(alpha=alpha, tol=tol), log=True)
results[("lasso", "il")]

100%|██████████| 10/10 [00:00<00:00, 35.10it/s]


{'bias': -871.0,
 'mae': 14344.0,
 'max_dev': 133423.0,
 'r2': 0.923,
 'rmse': 21064.0}

### 3. Improved Data with pre-selected Features

#### a) Normal Scale

In [38]:
grid_search.fit(X3, y3)
alpha = grid_search.best_params_["alpha"]
alpha

4

In [39]:
results[("lasso", "p")] = cross_validation(X3, y3, model=Lasso(alpha=alpha, tol=tol))
results[("lasso", "p")]

100%|██████████| 10/10 [00:00<00:00, 135.80it/s]


{'bias': -31.0,
 'mae': 23696.0,
 'max_dev': 167458.0,
 'r2': 0.821,
 'rmse': 32065.0}

#### b) Log Scale

In [40]:
grid_search.fit(X3, y3l)
alpha = grid_search.best_params_["alpha"]
alpha

0.00390625

In [41]:
results[("lasso", "pl")] = cross_validation(X3, y3l, model=Lasso(alpha=alpha, tol=tol), log=True)
results[("lasso", "pl")]

100%|██████████| 10/10 [00:00<00:00, 113.95it/s]


{'bias': -897.0,
 'mae': 16660.0,
 'max_dev': 145741.0,
 'r2': 0.899,
 'rmse': 24262.0}

## Ridge Regression

In [42]:
grid_search = GridSearchCV(
    estimator=Ridge(),
    param_grid={"alpha": [2 ** x for x in range(-8, 4)] + list(range(12, 65, 4))},
    cv=KFold(n_splits=4),
    n_jobs=-1,
)

### 1. Original Data

In [43]:
grid_search.fit(X1, y1)
alpha = grid_search.best_params_["alpha"]
alpha

0.125

In [44]:
results[("ridge", "o")] = cross_validation(X1, y1, model=Ridge(alpha=alpha))
results[("ridge", "o")]

100%|██████████| 10/10 [00:00<00:00, 33.96it/s]


{'bias': 152.0,
 'mae': 17064.0,
 'max_dev': 263561.0,
 'r2': 0.853,
 'rmse': 29970.0}

### 2. Improved Data

#### a) Normal Scale

In [45]:
grid_search.fit(X2, y2)
alpha = grid_search.best_params_["alpha"]
alpha

12

In [46]:
results[("ridge", "i")] = cross_validation(X2, y2, model=Ridge(alpha=alpha))
results[("ridge", "i")]

100%|██████████| 10/10 [00:00<00:00, 60.61it/s]


{'bias': -0.0,
 'mae': 15136.0,
 'max_dev': 132822.0,
 'r2': 0.918,
 'rmse': 21752.0}

#### b) Log Scale

In [47]:
grid_search.fit(X2, y2l)
alpha = grid_search.best_params_["alpha"]
alpha

2

In [48]:
results[("ridge", "il")] = cross_validation(X2, y2l, model=Ridge(alpha=alpha), log=True)
results[("ridge", "il")]

100%|██████████| 10/10 [00:00<00:00, 48.40it/s]


{'bias': -865.0,
 'mae': 12766.0,
 'max_dev': 119767.0,
 'r2': 0.936,
 'rmse': 19141.0}

### 3. Improved Data with pre-selected Features

#### a) Normal Scale

In [49]:
grid_search.fit(X3, y3)
alpha = grid_search.best_params_["alpha"]
alpha

4

In [50]:
results[("ridge", "p")] = cross_validation(X3, y3, model=Ridge(alpha=alpha))
results[("ridge", "p")]

100%|██████████| 10/10 [00:00<00:00, 267.05it/s]


{'bias': -20.0,
 'mae': 18391.0,
 'max_dev': 151814.0,
 'r2': 0.887,
 'rmse': 25686.0}

#### b) Log Scale

In [51]:
grid_search.fit(X3, y3l)
alpha = grid_search.best_params_["alpha"]
alpha

0.5

In [52]:
results[("ridge", "pl")] = cross_validation(X3, y3l, model=Ridge(alpha=alpha), log=True)
results[("ridge", "pl")]

100%|██████████| 10/10 [00:00<00:00, 262.02it/s]


{'bias': -1410.0,
 'mae': 16138.0,
 'max_dev': 144436.0,
 'r2': 0.907,
 'rmse': 23329.0}

## Random Forest

In [53]:
rf = RandomForestRegressor(
    n_estimators=500,
    n_jobs=-1, random_state=random_state
)

### 1. Original Data

In [54]:
results[("rf", "o")] = cross_validation(X1, y1, model=rf)
results[("rf", "o")]

100%|██████████| 10/10 [00:27<00:00,  2.77s/it]


{'bias': -26.0,
 'mae': 15322.0,
 'max_dev': 164505.0,
 'r2': 0.898,
 'rmse': 25354.0}

### 2. Improved Data

#### a) Normal Scale

In [55]:
results[("rf", "i")] = cross_validation(X2, y2, model=rf)
results[("rf", "i")]

100%|██████████| 10/10 [00:24<00:00,  2.48s/it]


{'bias': -26.0,
 'mae': 14817.0,
 'max_dev': 145413.0,
 'r2': 0.912,
 'rmse': 22565.0}

#### b) Log Scale

In [56]:
results[("rf", "il")] = cross_validation(X2, y2l, model=rf, log=True)
results[("rf", "il")]

100%|██████████| 10/10 [00:24<00:00,  2.47s/it]


{'bias': -2065.0,
 'mae': 14968.0,
 'max_dev': 150278.0,
 'r2': 0.908,
 'rmse': 23086.0}

### 3. Improved Data with pre-selected Features

#### a) Normal Scale

In [57]:
results[("rf", "p")] = cross_validation(X3, y3, model=rf)
results[("rf", "p")]

100%|██████████| 10/10 [00:17<00:00,  1.74s/it]


{'bias': -275.0,
 'mae': 16033.0,
 'max_dev': 149780.0,
 'r2': 0.899,
 'rmse': 24222.0}

#### b) Log Scale

In [58]:
results[("rf", "pl")] = cross_validation(X3, y3l, model=rf, log=True)
results[("rf", "pl")]

100%|██████████| 10/10 [00:16<00:00,  1.67s/it]


{'bias': -2378.0,
 'mae': 16319.0,
 'max_dev': 153478.0,
 'r2': 0.895,
 'rmse': 24746.0}

## Analysis of Results

This notebook did not focus on hyper-parameter optimization. Therefore, the predictions by Lasso, Ridge, and the Random Forest can potentially be improved by fine-graining the grid search even more.

In general, the manually "improved" data clearly outperform the data that were only cleaned with the minimum effort. Also, the result suggests to allow the model to select its features. The manually pre-selected features perform well but not as good as the full feature set.

In [59]:
def scores_by_source(source, score="rmse", *, ascending=True):
    rv = [
        (model, scores[score])
        for (model, data_source), scores in results.items()
        if data_source == source
    ]
    return sorted(rv, key=lambda x: x[1], reverse=(not ascending))

### Root Mean Squared Error

In [60]:
scores_by_source("o", "rmse")

[('rf', 25354.0), ('ridge', 29970.0), ('lasso', 33496.0), ('lm', 1902925384.0)]

In [61]:
scores_by_source("i", "rmse")

[('ridge', 21752.0), ('lm', 21864.0), ('rf', 22565.0), ('lasso', 24443.0)]

In [62]:
scores_by_source("il", "rmse")

[('ridge', 19141.0), ('lm', 19223.0), ('lasso', 21064.0), ('rf', 23086.0)]

In [63]:
scores_by_source("p", "rmse")

[('rf', 24222.0), ('ridge', 25686.0), ('lm', 25704.0), ('lasso', 32065.0)]

In [64]:
scores_by_source("pl", "rmse")

[('ridge', 23329.0), ('lm', 23341.0), ('lasso', 24262.0), ('rf', 24746.0)]

### R2

In [65]:
scores_by_source("o", "r2", ascending=False)

[('rf', 0.898), ('ridge', 0.853), ('lasso', 0.817), ('lm', -568953281.52)]

In [66]:
scores_by_source("i", "r2", ascending=False)

[('ridge', 0.918), ('lm', 0.917), ('rf', 0.912), ('lasso', 0.896)]

In [67]:
scores_by_source("il", "r2", ascending=False)

[('lm', 0.936), ('ridge', 0.936), ('lasso', 0.923), ('rf', 0.908)]

In [68]:
scores_by_source("p", "r2", ascending=False)

[('rf', 0.899), ('lm', 0.887), ('ridge', 0.887), ('lasso', 0.821)]

In [69]:
scores_by_source("pl", "r2", ascending=False)

[('lm', 0.907), ('ridge', 0.907), ('lasso', 0.899), ('rf', 0.895)]